In [8]:
import pandas as pd
import json

# Load the JSON file
with open("data.json", "r", encoding="utf-8") as file:
    data = json.load(file)  # Convert JSON to a Python list

# Convert list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Drop unwanted columns
df = df.drop(columns=["sticky", "terms", "has_deals"])

# Function to clean text (removes brackets, quotes)
def clean_text(value):
    if isinstance(value, list):  # Convert lists to comma-separated strings
        value = ", ".join(value)
    if isinstance(value, str):  # Remove unwanted characters
        return value.replace("[", "").replace("]", "").replace("'", "").replace('"', "")
    return value  # Return unchanged if not a string or list


# Apply cleaning to all columns
df = df.applymap(clean_text)

C:\Users\jaime\AppData\Local\Temp\ipykernel_21164\1816716326.py:24: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(clean_text)


In [9]:
# Save to CSV
df.to_csv("data.csv", index=False, encoding="utf-8")

print("CSV file saved as data.csv ✅")


CSV file saved as data.csv ✅


In [7]:
print(df.head())

  entity_id                                              title  \
0      1161                      Copamarina Beach Resort & Spa   
1      1211  Wyndham Grand Rio Mar Rainforest, Beach & Golf...   
2      1236               El Conquistador Resort - Puerto Rico   
3      1261                  Hilton Ponce Golf & Casino Resort   
4      1266        San Juan Marriott Resort & Stellaris Casino   

                                          categories   accommodations  \
0  Accommodation On Site, Boat Rentals & Charters...          Resorts   
1  Hotels, Accommodation On Site, Convention Cent...  Resorts, Hotels   
2  Ceremony Venues, Hotels, Accommodation On Site...  Resorts, Hotels   
3  Meeting Rooms, Ceremony Venues, Convention Cen...  Resorts, Hotels   
4  Meeting Rooms, Ceremony Venues, Hotels, Casino...  Resorts, Hotels   

                  cities regions locations  \
0                Guánica    West      None   
1             Río Grande    East      None   
2                Fajardo  

In [8]:
import pandas as pd
import json

# Load the JSON file
with open("food_drinks.json", "r", encoding="utf-8") as file:
    data = json.load(file)  # Convert JSON to a Python list

# Convert list of dictionaries to a DataFrame
food_df = pd.DataFrame(data)

# Save to CSV
food_df.to_csv("food_drinks.csv", index=False, encoding="utf-8")

print("CSV file saved as data.csv ✅")


CSV file saved as data.csv ✅


In [2]:
import pandas as pd
import json

# Load the JSON file
with open("hotels.json", "r", encoding="utf-8") as file:
    data = json.load(file)  # Convert JSON to a Python list

# Convert list of dictionaries to a DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv("hotels.csv", index=False, encoding="utf-8")

print("CSV file saved as hotel.csv ✅")

CSV file saved as hotel.csv ✅


In [3]:
print(df.head())

      @type  position                                                url
0  ListItem         0  https://www.discoverpuertorico.com/profile/car...
1  ListItem         1  https://www.discoverpuertorico.com/profile/dou...
2  ListItem         2  https://www.discoverpuertorico.com/profile/cop...
3  ListItem         3  https://www.discoverpuertorico.com/profile/cou...
4  ListItem         4  https://www.discoverpuertorico.com/profile/emb...


In [23]:
import requests
from bs4 import BeautifulSoup

# URL of the page you are scraping
url = "https://www.discoverpuertorico.com/profile/barrachina-restaurant/233"

# Fetch the page
response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
response.raise_for_status()

# Parse HTML
soup = BeautifulSoup(response.text, "html.parser")

# Print the full HTML of the page (or a smaller section if it's too big)
print(soup.prettify()[:5000])  # Print the first 5000 characters


<!DOCTYPE html>
<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
 <head>
  <meta charset="utf-8"/>
  <script type="text/javascript">
   dataLayer = [{"content_type":"profile","listing_id":"233","parent_term":"Dining"}];
  </script>
  <script type="application/ld+json">
   {
    "@context": "https://schema.org",
    "@type": "LocalBusiness",
    "name": "Barrachina Restaurant",
    "image": [
        "https://www.discoverpuertorico.com/sites/default/files/styles/large/public/listing_images/profile/233/191-3-15058_jpeg0-3dfc79875056a36_3dfc7d5a-5056-a36a-081d87aa5c540491.jpg?itok=GEZQU7Gv",
        "https://www.discoverpuertorico.com/sites/def

In [29]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import random

# Set up Selenium WebDriver
options = Options()
options.add_argument("--headless")  # Run in headless mode (no UI)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Load the CSV file with URLs
df = pd.read_csv("food_drinks.csv", header=None, names=["ListItem", "Index", "URL"])

# Drop the first row
df = df.drop(index=0).reset_index(drop=True)

# Function to scrape JSON-LD structured data and Google Maps URL
def scrape_page(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all JSON-LD structured data
        json_ld_scripts = soup.find_all("script", type="application/ld+json")

        json_data = None
        for script in json_ld_scripts:
            try:
                parsed_json = json.loads(script.string)
                if isinstance(parsed_json, list):
                    for obj in parsed_json:
                        if "@type" in obj:
                            json_data = obj
                            break
                elif "@type" in parsed_json:
                    json_data = parsed_json
                    break
            except json.JSONDecodeError:
                continue  # Skip invalid JSON

        if json_data:
            # Extract relevant fields
            place_type = json_data.get("@type", "N/A")
            name = json_data.get("name", "N/A")
            meta_description_tag = soup.find("meta", attrs={"name": "description"})
            description = meta_description_tag["content"] if meta_description_tag else "N/A"
            telephone = json_data.get("telephone", "N/A")
            website = json_data.get("url", "N/A")

            # Extract address details
            address_data = json_data.get("address", {})
            street = address_data.get("streetAddress", "N/A")
            city = address_data.get("addressLocality", "N/A")
            region = address_data.get("addressRegion", "N/A")
            postal_code = address_data.get("postalCode", "N/A")
            country = address_data.get("addressCountry", "N/A")
        else:
            place_type, name, description, telephone, website = "N/A", "N/A", "N/A", "N/A", "N/A"
            street, city, region, postal_code, country = "N/A", "N/A", "N/A", "N/A", "N/A"

        # Extract Google Maps URL using Selenium
        driver.get(url)

        time.sleep(random.uniform(1, 3))  # Random delay to avoid bot detection

        try:
            maps_link = driver.find_element(By.CSS_SELECTOR, "a[data-action='outbound_profile--get-directions']").get_attribute("href")
        except:
            maps_link = "N/A"

        


        print(f"Scraped: {name} | Google Maps URL: {maps_link}")

          # Rate limiting
        time.sleep(random.uniform(1, 4))  # Add a random delay between 3-6 seconds

        return place_type, name, description, street, city, region, postal_code, country, telephone, website, maps_link

    except requests.exceptions.RequestException as e:
        print(f"Error scraping {url}: {e}")
        return ("N/A",) * 11  # Return "N/A" for all fields if request fails

# Apply scraping function to each URL
df[["Type", "Name", "Description", "Street Address", "City", "Region", "Postal Code", "Country", "Telephone", "Website", "Google Maps URL"]] = df["URL"].apply(lambda url: pd.Series(scrape_page(url)))

# Save the updated dataframe
df.to_csv("food_drinks.csv", index=False, encoding="utf-8")

# Close Selenium WebDriver
driver.quit()

print("Scraping complete! Saved as food_drinks.csv ✅")


Scraped: Caribe Hilton | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJNWroByNvA4wRUBsD3LD2-ro&query=18.4631831000,-66.0856103000
Scraped: Copamarina Beach Resort & Spa | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJjZ-uUtpOA4wRC3eAHm0miT4&query=17.9533110000,-66.8796482000
Scraped: Barrachina Restaurant | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJt2zuY8duA4wR56__Tgqqm0M&query=18.4645020000,-66.1171904000
Scraped: Carabalí Rainforest Adventure Park | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJD1cjdNOeBIwRHFx_p_XREyI&query=18.2453145000,-65.9571458000
Scraped: Marmalade Restaurant & Wine Bar | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJYXyNtcZuA4wRNttsW6Mi8gQ&query=18.4656766000,-66.1133748000
Scraped: Emigrante Gastrobar | Google Maps URL: https://www.google.com/maps/search/?api=1&query=18.4434466000,-66.0268949000
Sc

In [ ]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import random

# Set up Selenium WebDriver
options = Options()
options.add_argument("--headless")  # Run in headless mode (no UI)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Load the CSV file with URLs
df = pd.read_csv("hotels.csv", header=None, names=["ListItem", "Index", "URL"])

# Drop the first row
df = df.drop(index=0).reset_index(drop=True)

# Function to scrape JSON-LD structured data and Google Maps URL
def scrape_page(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all JSON-LD structured data
        json_ld_scripts = soup.find_all("script", type="application/ld+json")

        json_data = None
        for script in json_ld_scripts:
            try:
                parsed_json = json.loads(script.string)
                if isinstance(parsed_json, list):
                    for obj in parsed_json:
                        if "@type" in obj:
                            json_data = obj
                            break
                elif "@type" in parsed_json:
                    json_data = parsed_json
                    break
            except json.JSONDecodeError:
                continue  # Skip invalid JSON

        if json_data:
            # Extract relevant fields
            place_type = json_data.get("@type", "N/A")
            name = json_data.get("name", "N/A")
            meta_description_tag = soup.find("meta", attrs={"name": "description"})
            description = meta_description_tag["content"] if meta_description_tag else "N/A"
            telephone = json_data.get("telephone", "N/A")
            website = json_data.get("url", "N/A")

            # Extract address details
            address_data = json_data.get("address", {})
            street = address_data.get("streetAddress", "N/A")
            city = address_data.get("addressLocality", "N/A")
            region = address_data.get("addressRegion", "N/A")
            postal_code = address_data.get("postalCode", "N/A")
            country = address_data.get("addressCountry", "N/A")
        else:
            place_type, name, description, telephone, website = "N/A", "N/A", "N/A", "N/A", "N/A"
            street, city, region, postal_code, country = "N/A", "N/A", "N/A", "N/A", "N/A"

        # Extract Google Maps URL using Selenium
        driver.get(url)

        time.sleep(random.uniform(1, 3))  # Random delay to avoid bot detection

        try:
            maps_link = driver.find_element(By.CSS_SELECTOR, "a[data-action='outbound_profile--get-directions']").get_attribute("href")
        except:
            maps_link = "N/A"

        


        print(f"Scraped: {name} | Google Maps URL: {maps_link}")

          # Rate limiting
        time.sleep(random.uniform(1, 4))  # Add a random delay between 3-6 seconds

        return place_type, name, description, street, city, region, postal_code, country, telephone, website, maps_link

    except requests.exceptions.RequestException as e:
        print(f"Error scraping {url}: {e}")
        return ("N/A",) * 11  # Return "N/A" for all fields if request fails

# Apply scraping function to each URL
df[["Type", "Name", "Description", "Street Address", "City", "Region", "Postal Code", "Country", "Telephone", "Website", "Google Maps URL"]] = df["URL"].apply(lambda url: pd.Series(scrape_page(url)))

# Save the updated dataframe
df.to_csv("hotels.csv", index=False, encoding="utf-8")

# Close Selenium WebDriver
driver.quit()

print("Scraping complete! Saved as hotels.csv ✅")


Scraped: Caribe Hilton | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJNWroByNvA4wRUBsD3LD2-ro&query=18.4631831000,-66.0856103000
Scraped: Doubletree by Hilton - San Juan | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJWezhytxuA4wRmNi83qdU-Vk&query=18.4510806000,-66.0644293000
Scraped: Copamarina Beach Resort & Spa | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJjZ-uUtpOA4wRC3eAHm0miT4&query=17.9533110000,-66.8796482000
Scraped: Courtyard by Marriott Isla Verde Beach Resort | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJq0MxYNBlA4wRDu4nlvRZ9wI&query=18.4432330000,-66.0087750000
Scraped: Embassy Suites by Hilton San Juan - Hotel & Casino | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJXQt7099lA4wRsDdD6EUYIzQ&query=18.4394497000,-66.0221968000
Scraped: Embassy Suites by Hilton Dorado del Mar Beach Resort | Google Maps URL: https:

In [12]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import random

# Set up Selenium WebDriver
options = Options()
options.add_argument("--headless")  # Run in headless mode (no UI)
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# Load the CSV file with URLs
df = pd.read_csv("data.csv")

# Drop the first row
df = df.drop(index=0).reset_index(drop=True)

# Function to scrape JSON-LD structured data and Google Maps URL
def scrape_page(url):
    try:
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all JSON-LD structured data
        json_ld_scripts = soup.find_all("script", type="application/ld+json")

        json_data = None
        for script in json_ld_scripts:
            try:
                parsed_json = json.loads(script.string)
                if isinstance(parsed_json, list):
                    for obj in parsed_json:
                        if "@type" in obj:
                            json_data = obj
                            break
                elif "@type" in parsed_json:
                    json_data = parsed_json
                    break
            except json.JSONDecodeError:
                continue  # Skip invalid JSON

        if json_data:
            # Extract relevant fields
            place_type = json_data.get("@type", "N/A")
            name = json_data.get("name", "N/A")
            meta_description_tag = soup.find("meta", attrs={"name": "description"})
            description = meta_description_tag["content"] if meta_description_tag else "N/A"
            telephone = json_data.get("telephone", "N/A")
            website = json_data.get("url", "N/A")

            # Extract address details
            address_data = json_data.get("address", {})
            street = address_data.get("streetAddress", "N/A")
            city = address_data.get("addressLocality", "N/A")
            region = address_data.get("addressRegion", "N/A")
            postal_code = address_data.get("postalCode", "N/A")
            country = address_data.get("addressCountry", "N/A")
        else:
            place_type, name, description, telephone, website = "N/A", "N/A", "N/A", "N/A", "N/A"
            street, city, region, postal_code, country = "N/A", "N/A", "N/A", "N/A", "N/A"

        # Extract Google Maps URL using Selenium
        driver.get(url)

        time.sleep(random.uniform(1, 3))  # Random delay to avoid bot detection

        try:
            maps_link = driver.find_element(By.CSS_SELECTOR, "a[data-action='outbound_profile--get-directions']").get_attribute("href")
        except:
            maps_link = "N/A"

        


        print(f"Scraped: {name} | Google Maps URL: {maps_link}")

          # Rate limiting
        time.sleep(random.uniform(1, 3))  # Add a random delay between 3-6 seconds

        return place_type, name, description, street, city, region, postal_code, country, telephone, website, maps_link

    except requests.exceptions.RequestException as e:
        print(f"Error scraping {url}: {e}")
        return ("N/A",) * 11  # Return "N/A" for all fields if request fails

# Apply scraping function to each URL
df[["Type", "Name", "Description", "Street Address", "City", "Region", "Postal Code", "Country", "Telephone", "Website", "Google Maps URL"]] = df["url"].apply(lambda url: pd.Series(scrape_page(url)))

# Save the updated dataframe
df.to_csv("thing_to_do.csv", index=False, encoding="utf-8")

# Close Selenium WebDriver
driver.quit()

print("Scraping complete! Saved as thing_to_do.csv ✅")


Scraped: Wyndham Grand Rio Mar Rainforest, Beach & Golf Resort | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJT4XxK0yZBIwRRfl1CbK81pY&query=18.3890790000,-65.7566780000
Scraped: El Conquistador Resort - Puerto Rico | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJ9xJQsBmXBIwRDK1TVXeQdNw&query=18.3584921000,-65.6283079000
Scraped: Hilton Ponce Golf & Casino Resort | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJswW1_g7WHIwRyuhT0ZLiybI&query=17.9713371000,-66.6023985000
Scraped: San Juan Marriott Resort & Stellaris Casino | Google Maps URL: https://www.google.com/maps/search/?api=1&query_place_id=ChIJsyyTEThvA4wRQ-WmN1ZtAEk&query=18.4564341000,-66.0701624000
Scraped: Group Services, Inc. | Google Maps URL: https://www.google.com/maps/search/?api=1&query=18.4569296000,-66.0712538000
Scraped: Caribbean Trading Company, Inc. - Río Grande | Google Maps URL: https://www.google.com/maps/search/?api=

In [15]:
df = pd.read_csv("hotels.csv")
print(df.head())

   Index                                                URL   Type  \
0      0  https://www.discoverpuertorico.com/profile/car...  Hotel   
1      1  https://www.discoverpuertorico.com/profile/dou...  Hotel   
2      2  https://www.discoverpuertorico.com/profile/cop...  Hotel   
3      3  https://www.discoverpuertorico.com/profile/cou...  Hotel   
4      4  https://www.discoverpuertorico.com/profile/emb...  Hotel   

                                                Name  \
0                                      Caribe Hilton   
1                    Doubletree by Hilton - San Juan   
2                      Copamarina Beach Resort & Spa   
3      Courtyard by Marriott Isla Verde Beach Resort   
4  Embassy Suites by Hilton San Juan - Hotel & Ca...   

                                         Description  \
0  The ever-iconic Caribe Hilton is situated on a...   
1  DoubleTree by Hilton Hotel San Juan is located...   
2  Set on a half-mile of private beach, amidst 20...   
3  Splash, relax a

In [14]:
import pandas as pd
import json

# Load CSV
csv_file = "things_to_do.csv"  # Change to your file path
df = pd.read_csv(csv_file)

# Drop unnecessary columns
df = df.drop(columns=["Unnamed: 15"], errors="ignore")  

# Convert CSV to JSON format
json_data = []
for _, row in df.iterrows():
    json_data.append({
        "id": row["index"],
        "name": row["Name"],
        "description": row["Description"],
        "type": row["Type"].split(", ") if pd.notna(row["Type"]) else [],
        "accommodations": row["accommodations"].split(", ") if pd.notna(row["accommodations"]) else [],
        "region": row["Region"],
        "location": {
            "street_address": row["Street Address"],
            "city": row["City"],
            "postal_code": str(row["Postal Code"]),
            "country": row["Country"]
        },
        "contact": {
            "telephone": row["Telephone"],
            "website": row["Website"],
            "google_maps_url": row["Google Maps URL"]
        },
        "photo": row["photo"],
        "url": row["url"]
    })

# Save to JSON file
json_file = "things_to_do.json"
with open(json_file, "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=4)

print(f"JSON file saved as {json_file}")


JSON file saved as things_to_do.json


In [18]:
import pandas as pd
import json

# Load CSV
csv_file = "food_drinks.csv"  # Change to your file path
df = pd.read_csv(csv_file)

# Replace NaN values with None (for JSON null values)
df = df.where(pd.notna(df), None)

# Convert CSV to JSON format
json_data = []
for _, row in df.iterrows():
    json_data.append({
        "id": row["Index"],
        "name": row["Name"],
        "description": row["Description"],
        "type": row["Type"],
        "region": row["Region"],
        "location": {
            "street_address": row["Street Address"],
            "city": row["City"],
            "postal_code": str(row["Postal Code"]) if row["Postal Code"] else None,
            "country": row["Country"]
        },
        "contact": {
            "telephone": row["Telephone"] if row["Telephone"] else None,
            "website": row["Website"] if row["Website"] else None,
            "google_maps_url": row["Google Maps URL"] if row["Google Maps URL"] else None
        },
        "url": row["URL"]
    })

# Save to JSON file
json_file = "food_drinks.json"
with open(json_file, "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=4)

print(f"JSON file saved as {json_file}")


JSON file saved as food_drinks.json
